In [4]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [98]:
from geopy.geocoders import Nominatim

In [99]:
from urllib.request import urlopen

In [100]:
import json

In [101]:
import requests

In [102]:
import pandas as pd

In [103]:
from math import radians, sin, cos, sqrt, atan2

In [104]:
def get_current_location():
    url = 'https://ipinfo.io/json'
    response = urlopen(url)
    data = json.load(response)
    return data

In [105]:
latitude, longitude = get_current_location()['loc'].split(',')

In [106]:
def get_place_info(api_key, query, latitude, longitude):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    
    params = {
        'query': query,
        'location': f'{latitude},{longitude}',
        'radius': '1000',
        'key': api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

In [107]:
def get_place_details(api_key, place_id):
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
    
    params = {
        'place_id': place_id,
        'fields': 'name,address_component,rating,website,opening_hours,user_ratings_total',
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    return data

In [108]:
def calculate_distance(lat1, lon1, lat2, lon2):

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    radius_of_earth = 6371.0  
    distance = radius_of_earth * c

    return distance

In [109]:
def calculate_weighted_score(rating, num_reviews):

    weight_rating = 0.6
    weight_num_reviews = 0.4

    weighted_score = (weight_rating * rating) + (weight_num_reviews * (num_reviews / 1000))  # Dividing by 1000 for normalization

    return weighted_score

In [118]:
try:
    user_location = [latitude, longitude]
    food_trucks_info = get_place_info(api_key, query, user_location[0], user_location[1])

    food_trucks_list = []

    for food_truck in food_trucks_info['results']:
        name = food_truck['name']
        address = food_truck['formatted_address']
        place_id = food_truck['place_id']

        details = get_place_details(api_key, place_id)

        rating = details['result'].get('rating', 'N/A')
        num_reviews = details['result'].get('user_ratings_total', 'N/A')
        website = details['result'].get('website', 'N/A')
        opening_hours = details['result']['opening_hours']['weekday_text'] if 'opening_hours' in details['result'] else 'N/A'
        weighted_score = calculate_weighted_score(rating, num_reviews)

        distance = calculate_distance(float(user_location[0]), float(user_location[1]), float(food_truck['geometry']['location']['lat']), float(food_truck['geometry']['location']['lng']))
        #print((user_location[0], user_location[1], food_truck['geometry']['location']['lat'], food_truck['geometry']['location']['lng']))
        
        lat, long = float(food_truck['geometry']['location']['lat']), float(food_truck['geometry']['location']['lng'])
        
        food_trucks_list.append([name, address, rating, num_reviews, website, opening_hours, distance, weighted_score, lat, long])

    df = pd.DataFrame(food_trucks_list, columns=['Name', 'Address', 'Rating', 'Num Reviews', 'Website', 'Opening Hours', 'Distance', 'Weighted Score', 'Latitude', 'Longitude'])

    df = df.sort_values(by=['Distance', 'Weighted Score', 'Num Reviews'], ascending=[True, False, False])

    top_6_food_trucks = df.head(6)

    #print(top_6_food_trucks)

except Exception as e:
    print(f"Error occurred while obtaining user location: {e}")


In [120]:
top_6_food_trucks.sort_values(by=['Weighted Score'], ascending=[False]).iloc[:, :7]

,Name,Address,Rating,Num Reviews,Website,Opening Hours,Distance
5,Wicked Food Shack,"2070 S 22nd St, Lafayette, IN 47905, United St...",4.90,176,N/A,"[Monday: Closed, Tuesday: Closed, Wednesday: 1...",2.21
1,Tortas Food Truck,"311 Sagamore Pkwy N, Lafayette, IN 47905, Unit...",4.80,51,https://www.facebook.com/pages/category/Produc...,"[Monday: 10:00 AM – 2:30 PM, 5:00 – 8:00 PM, T...",1.64
0,Tacos Mexico Food Truck,"3109 South St, Lafayette, IN 47904, United States",4.60,11,http://www.tacosmexicolafayette.com/,N/A,1.17
7,The Taste Of Tippecanoe,"638 North St, Lafayette, IN 47901, United States",4.20,37,http://www.tasteoftippecanoe.org/,"[Monday: Closed, Tuesday: Closed, Wednesday: C...",1.33
2,Famous Frank's,"30 Pierce St, West Lafayette, IN 47906, United...",4.10,20,N/A,"[Monday: Closed, Tuesday: Closed, Wednesday: C...",2.97
3,Tastes of Chicago,"311 Sagamore Pkwy N, Lafayette, IN 47904, Unit...",4.00,4,N/A,"[Monday: 12:00 – 6:00 PM, Tuesday: 12:00 – 6:0...",1.52


In [121]:
map_center = [top_6_food_trucks.iloc[0]['Latitude'], top_6_food_trucks.iloc[0]['Longitude']]
m = folium.Map(location=map_center, zoom_start=14)

for index, row in top_6_food_trucks.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='blue')
    ).add_to(m)

path_coordinates = top_6_food_trucks[['Latitude', 'Longitude']].values.tolist()
folium.PolyLine(
    locations=path_coordinates,
    color='green'
).add_to(m)

In [113]:
m